In [2]:
import re
import numpy as np
import pandas as pd
from profanity_check import predict, predict_prob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#parameters
MIN_COMMENT_LENGTH = 5 ;   #Removes any comment that has less than # characters
MIN_COMMENT_WORDS = 2;    #Removes any comment that has less than # words
PROFANITY_PROB = 0.5;

#load dataset 
filename = './dataset/dataset2.csv'
data = pd.read_csv(filename)
data.head()
comments = np.array(data['userMessage'].tolist())

In [16]:
def length_filter(comments):
    """
    length based filter
    """
    flags_len=[]
    for comment in comments: 
        if len(comment) <= MIN_COMMENT_LENGTH:
            flags_len.append(True)
        else:
            flags_len.append(False)
    return flags_len 
    
def profanity_filter(comments):
    """
    profanity filter
    """
    flags_profanity = predict_prob(comments) > PROFANITY_PROB
    return flags_profanity


def dict_filter(comments):
    """
    dictionary based filter
    """
    path1 = './dataset/hindi.txt'
    path2 = './dataset/english.txt'
    path3 = './dataset/spam_ads.txt'
    path4 = './dataset/blacklist.txt'
    with open(path1) as f1, open(path2) as f2, open(path3) as f3, open(path4) as f4:
        content_hi = f1.read()
        content_en = f2.read()
        content_spam = f3.read()
        content_black = f4.read()

    words_hi = content_hi.split("\n")
    words_en = content_en.split("\n")
    words_spam = content_spam.split("\n")
    words_black = content_black.split("\n")

    for comment in comments: 
        for word in words_hi:
            pattern = re.compile(word)
            match1 = pattern.search(comment.lower())
            if match1 is not None :
                for w in comment.split(' '):
                    match2 = re.fullmatch(pattern, w.lower())
                    if match2 is not None :
                        print("{:-<20} {}".format(word, comment))

    print("________________________________________________________________________________________")  
    #nsfw english filter
    filter_words_en = []
    for comment in comments: 
        for word in words_en:
            pattern = re.compile(word)
            match1 = pattern.search("comment.lower()")
            if match1 is not None :
                for w in comment.split(' '):
                    match2 = re.fullmatch(pattern, w.lower())
                    if match2 is not None :
                        print("{:-<20} {}".format(word, comment))    
    print("________________________________________________________________________________________")                    
    #spam filter
    for comment in comments: 
        for word in words_spam:
            pattern = re.compile(word)
            match1 = pattern.search(comment.lower())
            if match1 is not None :
                for w in comment.split(' '):
                    match2 = re.fullmatch(pattern, w.lower())
                    if match2 is not None :
                        print("{:-<20} {}".format(word, comment))
                        
    print("________________________________________________________________________________________")  
    #blacklist filter
    for comment in comments: 
        for word in words_black:
            pattern = re.compile(word)
            match1 = pattern.search(comment.lower())
            if match1 is not None :
                for w in comment.split(' '):
                    match2 = re.fullmatch(pattern, w.lower())
                    if match2 is not None :
                        print("{:-<20} {}".format(word, comment))


def emoji_filter(comments):
    """
    emoji based filter
    """
    pass

def senti_filter(comments):
    """
    sentiment based filter
    """
    flags_senti = []
    for comment in comments: 
        score = analyser.polarity_scores(comment)
        if score['neg'] > 0.5:
            flags_senti.append(True)
        else:
            flags_senti.append(False)

    return comments[flags_senti]




In [19]:
print(comments[length_filter(comments)])
print(comments[profanity_filter(comments)])
print(dict_filter(comments))


['Wow' 'Good ' '???' 'Hi' 'Good' '45' '????' 'Hi' '?????' 'Hii' 'Mast'
 'Cool' 'Nice' 'Osm' 'TQ' 'Jnn' 'text' '??' 'Nice' 'Hi' 'wow' 'Hi' 'Hi'
 'Hii' '✅✅✅✅' 'Good' 'Nam0' 'Wow' 'Osm' 'osm' 'Nice' 'Sure' '???' 'Hiii'
 'hi' 'H' 'Wow' 'nice' '?☕' 'nice' 'Wow' 'Gv' 'Hi' 'nan' 'Wow!' 'tasty'
 'Nice' 'Hello' 'Nice' 'Wahi' 'G' '??' 'Good' 'Hi' 'Hello' 'cool' 'Hiiii'
 'Yeah' 'Nice' 'Yes' 'Nic' 'Hello' '☺️☺️' 'Nice' 'Nice' 'Hii' 'Hi' 'Super'
 '?' 'Brr' 'Hii' 'Nice' 'Dope!' 'Hii' 'Goof' 'Eii' 'Hello' 'waah' 'Woww'
 'Nice' 'Hii' 'Nice' 'Aye♥️' 'Ok' 'Hi' 'Hii' 'Hi' 'Wow' 'Nice' 'Wow'
 'Nice' '?' 'Hi' 'Hello' 'Hi' 'wow!' 'gr8 ?' 'Bolo' 'Soon' 'Hindi' 'OMG'
 'okay' 'Nice' '??' 'Nice' 'Hello' 'woow' 'Hi' 'Hi' '3' 'Oh' 'Nicee'
 'Good' 'Hi' 'Nice' 'Darun' 'Cute' 'Hiii' 'Hello' 'nice' 'yum!' 'Hii'
 'Hello' 'Hello' 'Hello' 'Hi' 'Hy' 'Super' 'Nice' 'Wow' 'Sure' 'Nice'
 'nice' 'Hii' 'Hiii' '??' 'Hii' 'hi' 'Hi' 'Hii' 'Good' 'Nice' 'Cc' 'hello'
 '?' 'Nice' 'Yes' 'Hi' 'Nice' 'Hi' 'न्य्स' 'Hiiii' 'Niyc' 'Wow' 

free---------------- Win a Free Photoshoot 
A.k.a GET RECOGNIZED BY CELEBRITY MAKE UP ARTIST

Every month we will draw one new winner from our Facebook Fans.
Mybridalmakeup.com
The winner will receive a free 2 hour photo-shoot , 
redeemable towards a shoot in Mumbai.
free---------------- Win a Free Photoshoot 
A.k.a GET RECOGNIZED BY CELEBRITY MAKE UP ARTIST

Every month we will draw one new winner from our Facebook Fans.
Mybridalmakeup.com
The winner will receive a free 2 hour photo-shoot , 
redeemable towards a shoot in Mumbai.
cost---------------- Hai Anuj.Budget to reach this place depends from where you are travelling from.This is in  Sikkim 22 State of India and if you want to reached this place From capital.city ie.from Gangtok the you have to Hired a Cab which cost at around 5k approx. for a day but this place is very far from Gangtok as it is in West Sikkim .On its way you will find other different tourist destination Like Yuksom Old Capital.of Sikkim,Rabdentse Ruin,Peeling fr

In [17]:
print(dict_filter(comments))

________________________________________________________________________________________
________________________________________________________________________________________
free---------------- Win a Free Photoshoot 
A.k.a GET RECOGNIZED BY CELEBRITY MAKE UP ARTIST

Every month we will draw one new winner from our Facebook Fans.
Mybridalmakeup.com
The winner will receive a free 2 hour photo-shoot , 
redeemable towards a shoot in Mumbai.
free---------------- Win a Free Photoshoot 
A.k.a GET RECOGNIZED BY CELEBRITY MAKE UP ARTIST

Every month we will draw one new winner from our Facebook Fans.
Mybridalmakeup.com
The winner will receive a free 2 hour photo-shoot , 
redeemable towards a shoot in Mumbai.
100%---------------- Buy genuine IELTS certificate Online – Need Real verified IELTS certificate for sale – Get original IELTS certificate without Exam – I want verifiable IELTS Certificate – Buy real registered IELTS Certificate – Original IELTS certificate for sale – Genuine certific

cost---------------- How much did it cost you overall? [[@35288]]
free---------------- Win a Free Photoshoot 
A.k.a GET RECOGNIZED BY CELEBRITY MAKE UP ARTIST

Every month we will draw one new winner from our Facebook Fans.
Mybridalmakeup.com
The winner will receive a free 2 hour photo-shoot , 
redeemable towards a shoot in Mumbai.
free---------------- Win a Free Photoshoot 
A.k.a GET RECOGNIZED BY CELEBRITY MAKE UP ARTIST

Every month we will draw one new winner from our Facebook Fans.
Mybridalmakeup.com
The winner will receive a free 2 hour photo-shoot , 
redeemable towards a shoot in Mumbai.
cost---------------- Hai Anuj.Budget to reach this place depends from where you are travelling from.This is in  Sikkim 22 State of India and if you want to reached this place From capital.city ie.from Gangtok the you have to Hired a Cab which cost at around 5k approx. for a day but this place is very far from Gangtok as it is in West Sikkim .On its way you will find other different tourist desti

In [13]:
for word in words_en:
    pattern = re.compile(word)
    match1 = pattern.search("comment.lower()")analyser = SentimentIntensityAnalyzer()

    


score = sentiment_analyzer("this is fucking amazing")
print(score)

for comment in filter_words_en: 
    score = sentiment_analyzer(comment)
    if score['compound'] > 0:
        print(comment, score)


SyntaxError: invalid syntax (<ipython-input-13-57a9f6fc1b57>, line 3)